In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ml2020spring-hw1/sample_submission.csv
/kaggle/input/ml2020spring-hw1/test.csv
/kaggle/input/ml2020spring-hw1/train.csv


In [2]:
#### 先来看看数据是什么样的
train_data = pd.read_csv('/kaggle/input/ml2020spring-hw1/train.csv',encoding = 'big5')
test_data = pd.read_csv('/kaggle/input/ml2020spring-hw1/test.csv',encoding = 'big5',names = ['id', '測項', '0', '1', '2', '3', '4', '5', '6', '7', '8'])
test_data['id'] = test_data['id'].str.split('_',expand = True)[1].astype('int')
#train_data.info()
#train_data.head(20)
test_data

,id,測項,0,1,2,3,4,5,6,7,8
0,0,AMB_TEMP,21,21,20,20,19,19,19,18,17
1,0,CH4,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.8
2,0,CO,0.39,0.36,0.36,0.4,0.53,0.55,0.34,0.31,0.23
3,0,NMHC,0.16,0.24,0.22,0.27,0.27,0.26,0.27,0.29,0.1
4,0,NO,1.3,1.3,1.3,1.3,1.4,1.6,1.2,1.1,0.9
...,...,...,...,...,...,...,...,...,...,...,...
4315,239,THC,1.8,1.8,1.8,1.8,1.7,1.7,1.7,1.7,1.7
4316,239,WD_HR,80,92,95,95,96,97,96,96,84
4317,239,WIND_DIREC,76,99,93,97,93,94,98,97,65
4318,239,WIND_SPEED,2.2,3.2,2.5,3.6,5,4.2,5.7,4.9,3.6


这个数据总感觉哪里有点奇怪 
* 特征到底是哪些?
感觉24小时该作为特征 但好像测项的每个指标才是特征 
* 关于测项的指标数据
    1. CH4好像一直都是1.8?等下可以看看所有的CH4是不是都是1.8
    2. RAINFALL是string 应该是是否下雨的意思 转换为0和1吧 NR应该就是没下雨那就0好了 其他的(下雨)1
    3. 过于细致的数据应该需要用区间来替代 不然会出现overfitting的情况吧?

In [3]:
#先看看CH4
ch4 = train_data[train_data['測項'].isin(['CH4'])]
ch4.head()
#看看RAINFALL
rainfall = train_data[train_data['測項'].isin(['RAINFALL'])]
#rainfall.info()
#把RAINFALL按照上面说的那样replace一下
train_data.replace("NR",0, inplace = True)
test_data.replace("NR",0, inplace = True)
train_data.head(18)
rainfall = train_data[train_data['測項'].isin(['RAINFALL'])]
#rainfall.info()
#
for num in range(0,23) :
    num = str(num)
    train_data[num] = train_data[num].astype(float)

train_data.head()
#先置换一下行和列
#train_data_reshaped = train_data.pivot_table(train_data,index=[u'0'],columns="測項")
#train_data_reshaped.head()
#columns=('idx','degree','weight','diameter')
train_data_re = pd.DataFrame()
for num in range(24):
    num = str(num)
    train_data_re = train_data_re.append(
                pd.DataFrame(train_data[num].values.reshape(1,-1), 
                index=['id_'+num], 
                columns=train_data['測項'].values)
                )
train_data_re.head(25)

#把无用数据删掉
train_data.head()
train_data_v2 = train_data.copy()
#删掉了测站
train_data_v2 = train_data_v2.drop(columns = '測站')
train_data_v2


,日期,測項,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,AMB_TEMP,14.00,14.00,14.00,13.00,12.00,12.00,12.00,12.00,...,22.00,22.00,21.00,19.00,17.00,16.00,15.00,15.00,15.00,15
1,2014/1/1,CH4,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,...,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.8
2,2014/1/1,CO,0.51,0.41,0.39,0.37,0.35,0.30,0.37,0.47,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,NMHC,0.20,0.15,0.13,0.12,0.11,0.06,0.10,0.13,...,0.10,0.13,0.14,0.23,0.18,0.12,0.10,0.09,0.10,0.08
4,2014/1/1,NO,0.90,0.60,0.50,1.70,1.80,1.50,1.90,2.20,...,2.50,2.20,2.50,2.30,2.10,1.90,1.50,1.60,1.80,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,2014/12/20,THC,1.80,1.80,1.80,1.80,1.80,1.70,1.70,1.80,...,1.80,1.80,2.00,2.10,2.00,1.90,1.90,1.90,2.00,2
4316,2014/12/20,WD_HR,46.00,13.00,61.00,44.00,55.00,68.00,66.00,70.00,...,59.00,308.00,327.00,21.00,100.00,109.00,108.00,114.00,108.00,109
4317,2014/12/20,WIND_DIREC,36.00,55.00,72.00,327.00,74.00,52.00,59.00,83.00,...,18.00,311.00,52.00,54.00,121.00,97.00,107.00,118.00,100.00,105
4318,2014/12/20,WIND_SPEED,1.90,2.40,1.90,2.80,2.30,1.90,2.10,3.70,...,2.30,2.60,1.30,1.00,1.50,1.00,1.70,1.50,2.00,2


In [4]:
#train_data进行转换
train_data_v2

train_data_new = pd.DataFrame()
for x in range(24):
    train_data_first = train_data_v2[['日期','測項',str(x)]].copy()
    train_data_first['日期'] = pd.to_datetime(train_data_first['日期']+' '+str(x)+':00:00')
    train_data_first = train_data_first.pivot(index = '日期',columns = '測項', values = str(x))
    train_data_new = pd.concat([train_data_new,train_data_first])
train_data_new = train_data_new.astype('float64').sort_index().reset_index().drop(['日期'], axis = 1)
train_data_new

測項,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,14.0,1.8,0.51,0.20,0.9,16.0,17.0,16.0,56.0,26.0,0.0,77.0,1.8,2.0,37.0,35.0,1.4,0.5
1,14.0,1.8,0.41,0.15,0.6,9.2,9.8,30.0,50.0,39.0,0.0,68.0,2.0,2.0,80.0,79.0,1.8,0.9
2,14.0,1.8,0.39,0.13,0.5,8.2,8.7,27.0,48.0,36.0,0.0,67.0,1.7,2.0,57.0,2.4,1.0,0.6
3,13.0,1.8,0.37,0.12,1.7,6.9,8.6,23.0,35.0,35.0,0.0,74.0,1.6,1.9,76.0,55.0,0.6,0.3
4,12.0,1.8,0.35,0.11,1.8,6.8,8.5,24.0,25.0,31.0,0.0,72.0,1.9,1.9,110.0,94.0,1.7,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5755,14.0,1.8,0.48,0.15,1.0,13.0,14.0,18.0,54.0,37.0,0.0,76.0,2.6,1.9,109.0,97.0,1.0,1.4
5756,13.0,1.8,0.47,0.16,1.5,13.0,15.0,14.0,66.0,28.0,0.0,80.0,2.2,1.9,108.0,107.0,1.7,1.3
5757,13.0,1.8,0.51,0.16,1.5,13.0,15.0,13.0,50.0,17.0,0.0,82.0,2.3,1.9,114.0,118.0,1.5,1.6
5758,13.0,1.8,0.57,0.19,1.1,13.0,14.0,13.0,32.0,24.0,0.0,84.0,2.3,2.0,108.0,100.0,2.0,1.8


In [5]:
#feature scaling for train_data
#(X-mean)/std
train_mean = train_data_new.mean().copy()
train_std = train_data_new.std().copy()
train_data_new1 = train_data_new.copy()
for liecolumn in train_data_new:
        train_data_new[liecolumn] = (train_data_new[liecolumn] - train_mean[liecolumn])/train_std[liecolumn]
        #print(liecolumn,train_data_new[liecolumn])
        
train_data_new

測項,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,-1.355677,0.779179,0.375919,0.569287,-0.541475,0.949327,0.627186,-0.850401,0.506851,0.275214,-0.098084,0.282219,-0.530081,0.881810,-1.246312,-1.303972,-0.842156,-1.141225
1,-1.355677,0.779179,0.066870,0.091480,-0.672929,-0.149654,-0.323041,-0.101878,0.278038,1.055407,-0.098084,-0.391365,-0.420006,0.881810,-0.797207,-0.839334,-0.466713,-0.764819
2,-1.355677,0.779179,0.005060,-0.099643,-0.716748,-0.311268,-0.468215,-0.262276,0.201767,0.875363,-0.098084,-0.466208,-0.585118,0.881810,-1.037426,-1.648226,-1.217599,-1.047123
3,-1.514655,0.779179,-0.056750,-0.195204,-0.190929,-0.521367,-0.481412,-0.476140,-0.293994,0.815348,-0.098084,0.057691,-0.640156,0.331872,-0.838984,-1.092773,-1.593042,-1.329428
4,-1.673634,0.779179,-0.118560,-0.290765,-0.147111,-0.537529,-0.494610,-0.422674,-0.675349,0.575288,-0.098084,-0.091994,-0.475043,0.331872,-0.483877,-0.680935,-0.560574,-1.047123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5755,-1.355677,0.779179,0.283204,0.091480,-0.497656,0.464482,0.231258,-0.743469,0.430580,0.935378,-0.098084,0.207377,-0.089780,0.331872,-0.494322,-0.649255,-1.217599,-0.294312
5756,-1.514655,0.779179,0.252299,0.187041,-0.278565,0.464482,0.363234,-0.957333,0.888206,0.395244,-0.098084,0.506748,-0.309930,0.331872,-0.504766,-0.543656,-0.560574,-0.388413
5757,-1.514655,0.779179,0.375919,0.187041,-0.278565,0.464482,0.363234,-1.010799,0.278038,-0.264920,-0.098084,0.656433,-0.254893,0.331872,-0.442100,-0.427496,-0.748295,-0.106109
5758,-1.514655,0.779179,0.561349,0.473725,-0.453838,0.464482,0.231258,-1.010799,-0.408401,0.155184,-0.098084,0.806119,-0.254893,0.881810,-0.504766,-0.617575,-0.278991,0.082094


In [6]:
tx = train_data_new.copy()
tx.columns = tx.columns + '_0'
for i in range(1,10):
    ty = train_data_new.copy()
    if i == 9:
        ty = ty[['PM2.5']]
        # 结果列不需要标准化，需要放大回去
        ty = ty * train_std['PM2.5'] + train_mean['PM2.5']
    ty.columns = ty.columns + '_' + str(i)
    for j in range(i):
        ty = ty.drop([j])
    tx = pd.concat([tx, ty.reset_index().drop(['index'], axis=1)], axis=1)

for i in range(12):
    for j in range(9):
        tx = tx.drop([480*(i+1)-9+j])
train_data = tx
train_data.describe()

測項,AMB_TEMP_0,CH4_0,CO_0,NMHC_0,NO_0,NO2_0,NOx_0,O3_0,PM10_0,PM2.5_0,...,PM2.5_8,RAINFALL_8,RH_8,SO2_8,THC_8,WD_HR_8,WIND_DIREC_8,WIND_SPEED_8,WS_HR_8,PM2.5_9
count,5652.000000,5652.000000,5652.000000,5652.000000,5652.000000,5652.000000,5652.000000,5652.000000,5652.000000,5652.000000,...,5652.000000,5652.000000,5652.000000,5652.000000,5652.000000,5652.000000,5652.000000,5652.000000,5652.000000,5652.000000
mean,0.001447,-0.000058,0.002074,-0.002610,0.005129,-0.001535,0.000253,-0.000522,-0.004793,-0.004590,...,-0.002562,0.000750,-0.013894,0.005757,-0.000990,0.004713,0.005549,0.008337,0.003826,21.374027
std,0.999257,1.004981,1.008012,0.998278,1.006198,1.002372,1.003812,1.003651,1.002963,1.003339,...,0.997342,1.008693,0.999491,1.005869,1.003396,1.003836,1.003166,1.001812,1.003796,16.614767
min,-5.536816,-15.186918,-1.571093,-1.341940,-1.417839,-1.636509,-1.933149,-1.705857,-1.628736,-1.345187,...,-1.345187,-0.098084,-3.310232,-2.401358,-11.216819,-1.631707,-1.673570,-2.156207,-1.611732,-1.000000
25%,-0.719762,-0.817431,-0.427610,-0.673011,-0.453838,-0.682982,-0.652981,-0.743469,-0.713485,-0.745039,...,-0.745039,-0.098084,-0.690736,-0.640156,-0.768003,-0.880761,-0.871014,-0.748295,-0.764819,9.000000
50%,0.075131,-0.019126,-0.149465,-0.290765,-0.234747,-0.246622,-0.296646,-0.155344,-0.179588,-0.204905,...,-0.204905,-0.098084,0.132534,-0.254893,-0.218066,-0.389879,-0.385256,-0.185131,-0.200211,18.000000
75%,0.711046,0.779179,0.190489,0.378164,0.071981,0.464482,0.363234,0.539714,0.506851,0.455259,...,0.455259,-0.098084,0.806119,0.350521,0.331872,0.602331,0.652259,0.565756,0.552601,29.250000
max,2.141854,2.375788,22.194821,11.081035,12.647813,5.797768,7.753892,10.644782,5.273788,5.436493,...,5.436493,36.079898,1.928760,10.587510,6.381187,2.127201,2.128011,5.071072,4.975368,112.000000


In [48]:
test_data_new = pd.DataFrame()
for i in range(9):
    test_data_slice = test_data[['id', '測項', str(i)]].copy()
    test_data_slice = test_data_slice.pivot(index='id', columns='測項', values=str(i))
    test_data_slice.columns = test_data_slice.columns + '_' + str(i)
    for j in range(18):
        test_data_slice.iloc[:,[j]] = (test_data_slice.iloc[:,[j]].replace('NR', '0').astype('float64') - train_mean[j]) / train_std[j]
    test_data_new = pd.concat([test_data_new, test_data_slice], axis=1)

test_data_new = test_data_new.replace('NR', '0').astype('float64').reset_index().drop(['id'], axis=1)
test_data_new

測項,AMB_TEMP_0,CH4_0,CO_0,NMHC_0,NO_0,NO2_0,NOx_0,O3_0,PM10_0,PM2.5_0,...,PM10_8,PM2.5_8,RAINFALL_8,RH_8,SO2_8,THC_8,WD_HR_8,WIND_DIREC_8,WIND_SPEED_8,WS_HR_8
0,-0.242826,-0.019126,0.005060,0.187041,-0.366202,1.110941,0.759162,0.005054,0.735664,0.695318,...,-0.675349,-1.045113,-0.098084,0.506748,-0.199855,0.331872,-0.546543,-0.585895,0.002591,0.082094
1,-1.355677,0.779179,-0.180370,-0.004081,-0.629111,-0.020362,-0.164670,-0.422674,-0.255859,-0.264920,...,-0.370265,-0.204905,-0.098084,-0.017151,-0.640156,0.331872,-0.650986,-0.659815,-0.842156,-0.482515
2,1.505939,0.779179,0.994018,1.524900,-0.497656,0.949327,0.627186,3.640740,2.413626,1.715571,...,1.269561,0.275214,-0.098084,-0.017151,0.295483,-0.218066,-1.047870,-1.272292,-1.405321,-0.576616
3,-1.037719,0.779179,1.179448,2.002707,0.203435,4.828080,4.058563,-0.690003,5.083110,3.816091,...,-1.209246,-1.045113,-0.098084,-0.541051,-0.860306,-0.218066,-0.577876,-0.226857,-0.936017,-0.482515
4,0.234110,-0.019126,-0.334895,-0.290765,-0.366202,-0.068846,-0.164670,-1.299515,0.125496,-0.264920,...,1.193290,0.515273,-0.098084,-0.840422,0.845859,-0.768003,1.510986,1.462735,0.284173,0.552601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,0.075131,1.577484,0.715874,1.047094,0.115799,0.787712,0.627186,-1.395754,2.451761,2.495764,...,0.583122,1.775586,-0.098084,1.180332,0.075333,-0.218066,-0.995649,-1.187813,-0.936017,-1.141225
236,0.393089,-0.019126,0.406824,0.760410,0.159617,1.110941,1.023114,2.731819,2.833116,1.655556,...,1.956000,3.215943,-0.098084,0.955804,-0.585118,0.331872,-0.462989,-0.511976,-0.185131,0.176195
237,0.393089,0.779179,0.684969,0.187041,-0.322384,0.787712,0.495210,-0.208810,1.574645,2.255705,...,1.193290,1.055407,-0.098084,-1.813377,0.130370,-0.218066,1.604985,1.452175,0.941199,1.211311
238,-1.832613,-0.019126,-0.211275,-0.577449,-0.278565,-0.796112,-0.732167,-1.010799,-1.094840,-0.564994,...,-0.637214,-1.285173,-0.098084,-1.738534,2.992324,3.631498,1.040992,1.156496,-1.029877,-1.141225


拼接数组
np.vstack():在竖直方向上堆叠
np.hstack():在水平方向上平铺

In [8]:
#在后面都要加上一个常数 方便后面做gradient descent的时候使用
train_x = train_data.drop(['PM2.5_9'],axis = 1)
train_y = train_data[['PM2.5_9']]
x = np.hstack((train_x.values,np.ones((np.size(train_x.values,0),1),'double')))
y = train_y.values
x
y

array([[30.],
       [41.],
       [44.],
       ...,
       [17.],
       [24.],
       [29.]])

In [19]:
# 预测方法
# LOSS function
def loss(x,y,data):
    return np.sum((y- x @ data)**2)

# 梯度下降
def gradientDescent(x,y,data):
    return ((train_x_x) @ data) - (train_x_y) +(1 * data)

np.size(x,y)
如果传入的参数只有一个，则返回矩阵的元素个数
如果传入的第二个参数是0，则返回矩阵的行数
如果传入的第二个参数是1，则返回矩阵的列数

In [43]:
data = np.random.random((np.size(x,1),1))
data
#学习速率
learning_rate = 0.00000006
regular_one = 1

#把训练数据分成四份 按照3:1的比例设置为 训练集和验证集
train_X = x[:4320]
train_Y = y[:4320]
vari_X = x[4320:]
vari_Y = y[4320:]

#GD =  gradientDescent(train_X,train_Y,data)
#把gradientdescent里的参数提取出来 这样就不用在迭代过程中反复计算 可以节约时间
train_x_x = train_X.T @ train_X
train_x_y = train_X.T @ train_Y

for i in range(2000001):
    data = data - learning_rate * gradientDescent(train_X,train_Y,data)
    if i %50000 == 0 :
        #输出训练集误差和验证集误差 
        print(i,loss(train_X,train_Y,data) / np.size(train_Y,0) , loss(vari_X,vari_Y,data) / np.size(train_Y,0))


print(np.size(data,0), np.size(data,1))

0 896.6286418908277 407.6332117463146
50000 37.14537622320114 11.081461990876756
100000 34.93573454267098 10.103281469746587
150000 34.13472310674731 9.788080110426838
200000 33.7437222019846 9.658561350921964
250000 33.53141236006944 9.60064137572505
300000 33.40994948091966 9.574707816393772
350000 33.337970768532166 9.563993767132333
400000 33.29398124842131 9.56059995058974
450000 33.266227629782115 9.56060306823883
500000 33.24808684172407 9.562022581745547
550000 33.2357497808671 9.563872623189802
600000 33.22699051903367 9.565686341119601
650000 33.22048946961182 9.567266942625974
700000 33.21545285894039 9.568554756941289
750000 33.21139549509177 9.569556029546307
800000 33.208015323069624 9.57030522611682
850000 33.2051202816279 9.570845642446555
900000 33.20258526786254 9.57121996717371
950000 33.20032657869257 9.57146617089609
1000000 33.198286574356096 9.571616161182744
1050000 33.196424366029845 9.571695798490413
1100000 33.19471008268655 9.571725515384333
1150000 33.19312

In [49]:
test_x = np.hstack((test_data_new.values, np.ones((np.size(test_data_new.values,0), 1), 'double')))
print(np.size(test_x,0), np.size(test_x,1))

test_y = test_x @ (data)
test_y



240 163


array([[  7.09785095],
       [ 18.31089817],
       [ 24.88914605],
       [  6.90189597],
       [ 27.36112704],
       [ 22.43618601],
       [ 24.32272421],
       [ 31.13426114],
       [ 16.81443266],
       [ 60.23685019],
       [ 12.04706594],
       [  8.91637239],
       [ 63.18886424],
       [ 53.75023751],
       [ 21.94993811],
       [ 12.23056297],
       [ 32.22227549],
       [ 65.91001235],
       [ -1.27098775],
       [ 17.05422211],
       [ 41.6390036 ],
       [ 71.80973995],
       [  8.78408177],
       [ 17.82168106],
       [ 14.94707596],
       [ 38.59821098],
       [ 14.46174678],
       [ 68.95183207],
       [  6.87264544],
       [ 55.94954733],
       [ 24.34351413],
       [  9.27139601],
       [  3.5114583 ],
       [ 18.66783813],
       [ 28.65788667],
       [ 37.32161247],
       [ 43.15898671],
       [ 30.47359698],
       [ 41.30997315],
       [ 35.65528196],
       [  7.11823212],
       [ 41.51506361],
       [ 30.80344435],
       [ 51

In [50]:
test_data_id = test_data['id']

submission = pd.DataFrame({
        "id": test_data_id.unique(),
        "value": test_y.T[0]
    })
submission.to_csv('/kaggle/working/submission.csv', index=False)